In [16]:
import numpy as np


def get_distance(x1, x2):
    sum = 0
    for i in range(len(x1)):
        sum += (x1[i] - x2[i]) ** 2
    return np.sqrt(sum)


def kmeans(X, k, max_iters):
  
    centroids = X[np.random.choice(range(len(X)), k, replace=True)]

    converged = False
    
    current_iter = 0

    while (not converged) and (current_iter < max_iters):

        cluster_list = [[] for i in range(len(centroids))]

        for x in X:  # Go through each data point
            distances_list = []
            for c in centroids:
                distances_list.append(get_distance(c, x))
            cluster_list[int(np.argmin(distances_list))].append(x)

        cluster_list = list((filter(None, cluster_list)))

        prev_centroids = centroids.copy()

        centroids = []

        for j in range(len(cluster_list)):
            centroids.append(np.mean(cluster_list[j], axis=0))

        pattern = np.abs(np.sum(prev_centroids) - np.sum(centroids))

        print('K-MEANS: ', int(pattern))

        converged = (pattern == 0)

        current_iter += 1

    return np.array(centroids), [np.std(x) for x in cluster_list]

In [17]:
class RBF:

    def __init__(self, X, y, tX, ty, num_of_classes,
                 k, std_from_clusters=True):
        self.X = X
        self.y = y

        self.tX = tX
        self.ty = ty

        self.number_of_classes = num_of_classes
        self.k = k
        self.std_from_clusters = std_from_clusters

    def convert_to_one_hot(self, x, num_of_classes):
        arr = np.zeros((len(x), num_of_classes))
        for i in range(len(x)):
            c = int(x[i])
            arr[i][c] = 1
        return arr

    def rbf(self, x, c, s):
        distance = get_distance(x, c)
        return 1 / np.exp(-distance / s ** 2)

    def rbf_list(self, X, centroids, std_list):
        RBF_list = []
        for x in X:
            RBF_list.append([self.rbf(x, c, s) for (c, s) in zip(centroids, std_list)])
        return np.array(RBF_list)
    
    def fit(self):

        self.centroids, self.std_list = kmeans(self.X, self.k, max_iters=1000)

        if not self.std_from_clusters:
            dMax = np.max([get_distance(c1, c2) for c1 in self.centroids for c2 in self.centroids])
            self.std_list = np.repeat(dMax / np.sqrt(2 * self.k), self.k)

        RBF_X = self.rbf_list(self.X, self.centroids, self.std_list)

        self.w = np.linalg.pinv(RBF_X.T @ RBF_X) @ RBF_X.T @ self.convert_to_one_hot(self.y, self.number_of_classes)

        RBF_list_tst = self.rbf_list(self.tX, self.centroids, self.std_list)

        self.pred_ty = RBF_list_tst @ self.w

        self.pred_ty = np.array([np.argmax(x) for x in self.pred_ty])

        diff = self.pred_ty - self.ty

        print('Accuracy: ', len(np.where(diff == 0)[0]) / len(diff))

In [18]:
from sklearn.datasets import load_iris, load_breast_cancer, load_digits
from sklearn.model_selection import KFold

## Iris

In [19]:
iris = load_iris()
X, y = iris.data, iris.target

In [20]:
kfold = KFold(n_splits=2, shuffle=True)
for train_idx, test_idx in kfold.split(X):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

In [21]:
RBF_CLASSIFIER = RBF(X_train, y_train, X_test, y_test, num_of_classes=10,
                     k=500, std_from_clusters=False)

RBF_CLASSIFIER.fit()

K-MEANS:  6030
K-MEANS:  0
Accuracy:  0.6266666666666667


## Breast Cancer

In [22]:
bcancer = load_breast_cancer()
X, y = bcancer.data, bcancer.target

In [23]:
kfold = KFold(n_splits=2, shuffle=True)
for train_idx, test_idx in kfold.split(X):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

In [24]:
RBF_CLASSIFIER = RBF(X_train, y_train, X_test, y_test, num_of_classes=10,
                     k=500, std_from_clusters=False)

RBF_CLASSIFIER.fit()

K-MEANS:  464425
K-MEANS:  45
K-MEANS:  0
Accuracy:  0.926056338028169


## Digit

In [25]:
digit = load_digits()
X, y = digit.data, digit.target

In [26]:
kfold = KFold(n_splits=2, shuffle=True)
for train_idx, test_idx in kfold.split(X):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

In [27]:
RBF_CLASSIFIER = RBF(X_train, y_train, X_test, y_test, num_of_classes=10,
                     k=500, std_from_clusters=False)

RBF_CLASSIFIER.fit()

K-MEANS:  34963
K-MEANS:  71
K-MEANS:  0
Accuracy:  0.9599109131403119
